In [47]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import date

In [48]:
df_old = pd.read_csv("brd_covid_19.csv",parse_dates=[7])

In [49]:
df_old = df_old.sort_values(["bundesland", "datum"])

df_old.drop_duplicates(inplace=True)

df_old = df_old.reset_index().drop("index",axis = 1)

In [50]:
today = date.today()

In [51]:
URL = "https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html"

In [52]:
page = requests.get(URL)

In [53]:
soup = BeautifulSoup(page.content, 'html.parser')

In [54]:
results = soup.find_all("tr")

In [55]:
bundesland = []
faelle = []
tod = []
neue_faelle=[]
faelle_per_capita=[]
for idx,elm in enumerate(results):
    if idx > 1:
        bundesland.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[0])))
        faelle.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[1])))
        neue_faelle.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[2])))
        faelle_per_capita.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[3])))
        tod.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[4])))
    else:
        pass

In [56]:
bundesland =  [elm.replace("\xad","") for elm in bundesland]
faelle = [int(elm.replace(".","")) for elm in faelle]
tod = [elm.replace(".","") for elm in tod]
neue_faelle = [int(elm.strip("+").replace(".","")) for elm in neue_faelle]
faelle_per_capita = [float(elm.replace(",",".")) for elm in faelle_per_capita]

In [57]:
lat={"Baden-Württemberg":48.5, "Bayern":48.9, "Berlin":52.5, "Brandenburg":52, "Bremen":53.1, "Hamburg":53.5, 
     "Hessen":50.7, "Mecklenburg-Vorpommern":53.8, "Niedersachsen":52.57, "Nordrhein-Westfalen":51.6,"Rheinland-Pfalz":50, 
     "Saarland":49.4, "Sachsen":51.05, "Sachsen-Anhalt":52, "Schleswig-Holstein":54.2, "Thüringen":50.85, "Gesamt":51.06}
long={"Baden-Württemberg":9, "Bayern":11.4, "Berlin":13.4, "Brandenburg":13.87, "Bremen":8.8, "Hamburg":10, 
      "Hessen":9, "Mecklenburg-Vorpommern":12.5,"Niedersachsen":10, "Nordrhein-Westfalen":7.5,"Rheinland-Pfalz":7.3,
      "Saarland":6.97,"Sachsen":13.5, "Sachsen-Anhalt":11.65,"Schleswig-Holstein":9.9, "Thüringen":11.13,"Gesamt":10}


In [58]:
df=pd.DataFrame(data=[bundesland,faelle,tod,neue_faelle,faelle_per_capita]).T
    

In [59]:
df["lat"]="NaN"
df["long"]="NaN"
df["date"] = today

In [60]:
df.columns = ["bundesland","faelle","tod","neue_faelle","faelle_per_capita","lat","long","datum"]

In [61]:
#df.tod = df.tod.replace(to_replace = "", value = "0", regex = True)

In [62]:
df.datum = pd.to_datetime(df.datum)

In [63]:
df.bundesland.values

array(['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen',
       'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen',
       'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen',
       'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen', 'Gesamt'],
      dtype=object)

In [64]:
df.bundesland.replace("Schleswig Holstein","Schleswig-Holstein", inplace=True)
lat2=[]
long2=[]
for elm in df.bundesland.values:
    lat2.append(lat[elm])
    long2.append(long[elm])

In [65]:
df["lat"] = lat2
df["long"] = long2

In [66]:
df

,bundesland,faelle,tod,neue_faelle,faelle_per_capita,lat,long,datum
0,Baden-Württemberg,3807,21,139,34,48.50,9.00,2020-03-22
1,Bayern,3650,21,690,28,48.90,11.40,2020-03-22
2,Berlin,1024,1,158,27,52.50,13.40,2020-03-22
3,Brandenburg,274,,20,11,52.00,13.87,2020-03-22
4,Bremen,165,,23,24,53.10,8.80,2020-03-22
5,Hamburg,872,,285,47,53.50,10.00,2020-03-22
6,Hessen,1175,2,95,19,50.70,9.00,2020-03-22
7,Mecklenburg-Vorpommern,172,,7,11,53.80,12.50,2020-03-22
8,Niedersachsen,1306,1,283,16,52.57,10.00,2020-03-22
9,Nordrhein-Westfalen,3545,6,3,20,51.60,7.50,2020-03-22


In [67]:
#df.lat = df.lat.astype("float")
#df.long = df.long.astype("float")
df.tod = df.tod.replace(to_replace = "", value = "0", regex = True)
df.tod = df.tod.astype("int64")
df.faelle = df.faelle.astype("int64")
df.neue_faelle = df.neue_faelle.astype("int64")
df.faelle_per_capita = df.faelle_per_capita.astype("float64")

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   bundesland         17 non-null     object        
 1   faelle             17 non-null     int64         
 2   tod                17 non-null     int64         
 3   neue_faelle        17 non-null     int64         
 4   faelle_per_capita  17 non-null     float64       
 5   lat                17 non-null     float64       
 6   long               17 non-null     float64       
 7   datum              17 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(3), object(1)
memory usage: 1.2+ KB


In [71]:
pd.concat([df_old,df]).reset_index().drop("index",axis=1)

,bundesland,faelle,tod,neue_faelle,faelle_per_capita,lat,long,datum
0,Baden-Württemberg,10,0,0,0.1,48.50,9.00,2020-02-28
1,Baden-Württemberg,14,0,4,0.1,48.50,9.00,2020-02-29
2,Baden-Württemberg,15,0,1,0.1,48.50,9.00,2020-03-01
3,Baden-Württemberg,19,0,4,0.2,48.50,9.00,2020-03-02
4,Baden-Württemberg,26,0,7,0.2,48.50,9.00,2020-03-03
...,...,...,...,...,...,...,...,...
359,Sachsen,606,0,39,15.0,51.05,13.50,2020-03-22
360,Sachsen-Anhalt,211,0,23,10.0,52.00,11.65,2020-03-22
361,Schleswig-Holstein,347,1,39,12.0,54.20,9.90,2020-03-22
362,Thüringen,216,0,29,10.0,50.85,11.13,2020-03-22


In [72]:
df=pd.concat([df_old,df]).reset_index().drop("index",axis=1)

In [73]:
df[df["bundesland"]=="Bayern"]

,bundesland,faelle,tod,neue_faelle,faelle_per_capita,lat,long,datum
23,Bayern,15,0,0,0.10,48.9,11.4,2020-02-28
24,Bayern,15,0,0,0.10,48.9,11.4,2020-02-29
25,Bayern,19,0,4,0.10,48.9,11.4,2020-03-01
26,Bayern,25,0,6,0.20,48.9,11.4,2020-03-02
27,Bayern,35,0,10,0.30,48.9,11.4,2020-03-03
28,Bayern,48,0,13,0.40,48.9,11.4,2020-03-04
29,Bayern,52,0,4,0.40,48.9,11.4,2020-03-05
30,Bayern,79,0,27,0.60,48.9,11.4,2020-03-06
31,Bayern,117,0,38,0.90,48.9,11.4,2020-03-07
32,Bayern,148,0,31,1.10,48.9,11.4,2020-03-08


In [74]:
df_old.to_csv ('old_data/brd_covid_19'+str(today)+".csv", index = False, header=True)

In [75]:
df.to_csv ('brd_covid_19.csv', index = False, header=True)